In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
import sys
sys.path.append('../')

In [3]:
from pyballmapper.ballmapper import BallMapper

In [5]:
print('loading jones')
jones_df = pd.read_csv('../data/parsed/Jones/Jones_upto_15_MIRRORS.csv')
print(jones_df.shape)

loading jones
(626462, 59)


In [ ]:
X = jones_df[jones_df.columns[8:]].to_numpy()
y = jones_df[['number_of_crossings', 'is_alternating', 'signature']]

In [ ]:
orbits = [] 

for i in range(0, len(X), 2):
    orbits.append([i, i+1])
    orbits.append([i+1, i])

In [ ]:
for eps in [50, 40, 30]:
    jones_BM = BallMapper(X, eps=eps, dbg=True)

    for node in jones_BM.Graph.nodes:
        jones_BM.Graph.nodes[node]['points covered'] = np.array(jones_BM.Graph.nodes[node]['points covered'])

    with open('pkl/jones_{}.pkl'.format(eps), 'wb') as f:
        pickle.dump(jones_BM, f)

## MoBM homfly

In [ ]:
print()
print('read homfly')
homfly_df = pd.read_csv('../data/parsed/HOMFLYPT/HomflyPt_upto_15_MIRRORS.csv')
print(homfly_df.shape)

In [ ]:
X_h = homfly_df[homfly_df.columns[6:]].to_numpy()

In [ ]:
from pyballmapper.ballmapper import mapper_on_BM

In [ ]:
for j_eps in [50, 30, 40]:
    print(j_eps)
    with open('pkl/jones_{}.pkl'.format(j_eps), 'rb') as f:
        jones_BM = pikle.load(jones_BM)

    for eps_db in [40, 30, 20]:
        print('    ', eps_db)
        mobm = mapper_on_BM(jones_BM.Graph, X_h, eps_db, min_samples=1, dbscan_metric='euclidean', sparse=False, dbg=True)

        with open('pkl/mobm_{}_{}.pkl'.format(j_eps, eps_db), 'wb') as f:
            pickle.dump(mobm, f)